In [125]:
# 사용할 모듈 불러오기 
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re
import xgboost

In [126]:
# 데이터 불러오기
items = pd.read_csv("./data/items.csv")
shops = pd.read_csv("./data/shops.csv")
train = pd.read_csv("./data/sales_train.csv")
test = pd.read_csv("./data/test.csv")
item_categories = pd.read_csv('./data/item_categories.csv')

In [127]:
#  date의 자료형을 object  -> dattime 으로 바꾸고 싶다. 
# 변환하는 과정  ->
train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")
train["date"]. head(3) 

0   2013-01-02
1   2013-01-03
2   2013-01-05
Name: date, dtype: datetime64[ns]

In [128]:
test.columns , train.columns

(Index(['ID', 'shop_id', 'item_id'], dtype='object'),
 Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
        'item_cnt_day'],
       dtype='object'))

In [129]:
test['date_block_num'] = 34
test['item_price'] = 1
test['item_cnt_day'] = 0
del train['date']
del test['ID']

In [130]:
pd.concat([test,train])

,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,5,5037,34,1.0,0.0
1,5,5320,34,1.0,0.0
2,5,5233,34,1.0,0.0
3,5,5232,34,1.0,0.0
4,5,5268,34,1.0,0.0
...,...,...,...,...,...
2935844,25,7409,33,299.0,1.0
2935845,25,7460,33,299.0,1.0
2935846,25,7459,33,349.0,1.0
2935847,25,7440,33,299.0,1.0


In [131]:
pd.concat([test,train]).sort_values('date_block_num').reset_index().drop('index',axis = 1)

,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,16,14041,0,149.0,1.0
1,25,21857,0,149.0,1.0
2,25,21857,0,149.0,1.0
3,25,21857,0,149.0,1.0
4,25,21964,0,1199.0,1.0
...,...,...,...,...,...
3150044,57,18454,34,1.0,0.0
3150045,57,18027,34,1.0,0.0
3150046,57,18049,34,1.0,0.0
3150047,57,18589,34,1.0,0.0


In [132]:
train_df = pd.concat([test,train]).sort_values('date_block_num').reset_index().drop('index',axis = 1)
train_df

,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,16,14041,0,149.0,1.0
1,25,21857,0,149.0,1.0
2,25,21857,0,149.0,1.0
3,25,21857,0,149.0,1.0
4,25,21964,0,1199.0,1.0
...,...,...,...,...,...
3150044,57,18454,34,1.0,0.0
3150045,57,18027,34,1.0,0.0
3150046,57,18049,34,1.0,0.0
3150047,57,18589,34,1.0,0.0


In [133]:
train_df = train_df.pivot_table(index = ['date_block_num','shop_id','item_id','item_price'] , values=['item_cnt_day'],aggfunc='sum')
train_df.rename(columns = {'item_cnt_day':'item_cnt_month'}, inplace = True)
train_df

item_cnt_month
date_block_num shop_id item_id item_price                
0              0       32      221.0                  6.0
                       33      347.0                  3.0
                       35      247.0                  1.0
                       43      221.0                  1.0
                       51      127.0                  1.0
...                                                   ...
34             59      22162   1.0                    0.0
                       22163   1.0                    0.0
                       22164   1.0                    0.0
                       22166   1.0                    0.0
                       22167   1.0                    0.0

[1953222 rows x 1 columns]

In [134]:
train_df = train_df.reset_index()
train_df

,date_block_num,shop_id,item_id,item_price,item_cnt_month
0,0,0,32,221.0,6.0
1,0,0,33,347.0,3.0
2,0,0,35,247.0,1.0
3,0,0,43,221.0,1.0
4,0,0,51,127.0,1.0
...,...,...,...,...,...
1953217,34,59,22162,1.0,0.0
1953218,34,59,22163,1.0,0.0
1953219,34,59,22164,1.0,0.0
1953220,34,59,22166,1.0,0.0


In [135]:
x_train = train_df[train_df['date_block_num'] < 33]
x_train = x_train.drop('item_cnt_month', axis = 1)
y_train = train_df[train_df['date_block_num'] < 33]['item_cnt_month']

x_valid = train_df[train_df['date_block_num'] == 33]
x_valid = x_valid.drop('item_cnt_month', axis = 1)
y_valid = train_df[train_df['date_block_num'] == 33]['item_cnt_month']

x_test = train_df[train_df['date_block_num'] == 34]
x_test = x_test.drop('item_cnt_month', axis = 1)
y_test = train_df[train_df['date_block_num'] == 34]['item_cnt_month']

In [136]:
x_valid

,date_block_num,shop_id,item_id,item_price
1705824,33,2,31,399.0
1705825,33,2,486,300.0
1705826,33,2,787,420.0
1705827,33,2,794,3300.0
1705828,33,2,968,58.0
...,...,...,...,...
1739017,33,59,22087,119.0
1739018,33,59,22088,119.0
1739019,33,59,22091,179.0
1739020,33,59,22100,629.0


In [145]:
from lightgbm import LGBMRegressor

In [138]:
model=lgb(
        n_estimators=200,
        learning_rate=0.03,
        num_leaves=32,
        colsample_bytree=0.9497036,
        subsample=0.8715623,
        max_depth=8,
        reg_alpha=0.04,
        reg_lambda=0.073,
        min_split_gain=0.0222415,
        min_child_weight=40)

In [139]:
print('Training time, it is...')
model.fit(x_train, y_train,)

Training time, it is...


LGBMRegressor(colsample_bytree=0.9497036, learning_rate=0.03, max_depth=8,
              min_child_weight=40, min_split_gain=0.0222415, n_estimators=200,
              num_leaves=32, reg_alpha=0.04, reg_lambda=0.073,
              subsample=0.8715623)

In [140]:
x_train

,date_block_num,shop_id,item_id,item_price
0,0,0,32,221.0
1,0,0,33,347.0
2,0,0,35,247.0
3,0,0,43,221.0
4,0,0,51,127.0
...,...,...,...,...
1705819,32,59,22087,119.0
1705820,32,59,22088,119.0
1705821,32,59,22091,179.0
1705822,32,59,22100,629.0


In [141]:
x_test

,date_block_num,shop_id,item_id,item_price
1739022,34,2,30,1.0
1739023,34,2,31,1.0
1739024,34,2,32,1.0
1739025,34,2,33,1.0
1739026,34,2,38,1.0
...,...,...,...,...
1953217,34,59,22162,1.0
1953218,34,59,22163,1.0
1953219,34,59,22164,1.0
1953220,34,59,22166,1.0


In [143]:
# Get the test set predictions and clip values to the specified range
y_pred = model.predict(x_test).clip(0, 20)

# Create the submission file and submit!
preds = pd.DataFrame(y_pred, columns=['item_cnt_month'])
preds.to_csv('submission.csv',index_label='ID')